<a href="https://colab.research.google.com/github/wgova/kmeans-clustering/blob/master/jse_clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
import numpy as np
from scipy.cluster.vq import kmeans,vq
import pandas as pd
import pandas_datareader as dr
from math import sqrt
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import os

In [0]:
PATH = '/content/drive/My Drive/git_projects/jse_blue_tickers'
url = "https://www.sashares.co.za/shares-list/#gs.5pxra9"
jse = pd.read_html(url)
df_jse = jse[0]

In [0]:
df_jse.rename(columns={"JSE Code":"jse_code",
                       "Short Name":"name",
                       "Market Cap":"market_cap",
                       "Next Results":"results"},inplace=True)
df_jse[['jse_code','name']].astype("category")
print(df_jse.columns)

exclude = ["BIK","IMCB22","CHP","CVW","ECSB10","ECSB11","ERN","ESCB9",'FDP',"FSE","GRF","HWA","MED","MRI","RDI","TCS"]
jse_list = df_jse[~df_jse['jse_code'].isin(exclude)]

jse = list(jse_list["jse_code"])
for i in range(0,len(jse)):
  jse[i]=jse[i]+".JO"
jse[0]

In [0]:
%%time
tickers = jse
prices_list = []
for ticker in tickers:
    try:
        prices = dr.DataReader(ticker,'yahoo','01/01/2009')['Adj Close']
        prices = pd.DataFrame(prices)
        prices.columns = [ticker]
        prices_list.append(prices)
    except:
        pass
    prices_df = pd.concat(prices_list,axis=1)
prices_df.sort_index(inplace=True)

In [0]:
# dropping tickers missing 1 year of data
prices_df.dropna(thresh=250,axis=1)
# prices = pd.melt(prices_df).rename(columns={"variable":"ticker","value":"price"})
# prices.ticker.ravel()

In [0]:
## remove all columns >=0.1
# exclude_prices = prices_df.loc[:, prices_df.isnull().mean() < .001]
# prices.isnull().sum()

In [0]:
returns = prices_df.pct_change().mean() * 252
returns = pd.DataFrame(returns)
returns.columns = ['Returns']
returns['Volatility'] = prices_df.pct_change().std() * sqrt(252)
returns.dropna(axis=0,inplace=True)
#format the data as a numpy array to feed into the K-Means algorithm
returns.head()

In [0]:
data = np.asarray([np.asarray(returns['Returns']),np.asarray(returns['Volatility'])]).T
X = data
distorsions = []
for k in range(2, 20):
    k_means = KMeans(n_clusters=k)
    k_means.fit(X)
    distorsions.append(k_means.inertia_)
fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')
plt.show()

In [0]:
returns.isnull().sum()

In [0]:
# computing K-Means with K = 5 (5 clusters)
centroids,_ = kmeans(data,5)
# assign each sample to a cluster
idx,_ = vq(data,centroids)
# some plotting using numpy's logical indexing
plot(data[idx==0,0],data[idx==0,1],'ob',
     data[idx==1,0],data[idx==1,1],'oy',
     data[idx==2,0],data[idx==2,1],'or',
     data[idx==3,0],data[idx==3,1],'og',
     data[idx==4,0],data[idx==4,1],'om')
plot(centroids[:,0],centroids[:,1],'sg',markersize=8)
plt.show()

In [0]:
details = [(name,cluster) for name, cluster in zip(returns.index,idx)]
# for detail in details:
#     print(detail)

d = pd.DataFrame(details,columns=['names','cluster'])
d['names'].astype('category')

x = returns.reset_index()
x = x.rename(columns={'index':'names'})
x.names.astype('category')
z = pd.merge(x,d,how='inner', on='names')
import seaborn as sns
sns.pairplot(z, hue='cluster')
piv = pd.pivot(z,columns='cluster',values='Returns')
piv.plot(subplots=True)